In [2]:
import pandas as pd
import numpy as np
from dateutil import parser

# 1. Load Data
# Ensure you use the cleaned file from the previous step if available
file_name = "SLU Opportunity Wise Data-1710158595043 - SLU Opportunity Wise Data-1710158595043.csv"
try:
    df = pd.read_csv(file_name)
except:
    # Fallback if csv not found (adjust extension if needed)
    df = pd.read_excel("SLU Opportunity Wise Data-1710158595043.xlsx")

# 2. Date Preprocessing (Required for feature engineering)
date_cols = ['Learner SignUp DateTime', 'Apply Date']

def robust_date_parse(date_str):
    if pd.isna(date_str): return pd.NaT
    try: return parser.parse(str(date_str))
    except: return pd.NaT

for col in date_cols:
    try:
        df[col] = pd.to_datetime(df[col], format='mixed', errors='coerce')
    except ValueError:
        df[col] = df[col].apply(robust_date_parse)

# Remove rows without an Apply Date (cannot forecast without a timestamp)
df = df.dropna(subset=['Apply Date']).sort_values('Apply Date')

# --- 3. Feature Engineering ---

# A. Time-Derived Variables
df['Apply_Year'] = df['Apply Date'].dt.year
df['Apply_Month'] = df['Apply Date'].dt.month
df['Apply_Week'] = df['Apply Date'].dt.isocalendar().week
df['Apply_DayOfWeek'] = df['Apply Date'].dt.dayofweek  # 0=Monday, 6=Sunday
df['Apply_Hour'] = df['Apply Date'].dt.hour
df['Is_Weekend'] = df['Apply_DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)

# Seasonality
def get_season(month):
    if month in [12, 1, 2]: return 'Winter'
    elif month in [3, 4, 5]: return 'Spring'
    elif month in [6, 7, 8]: return 'Summer'
    else: return 'Fall'
df['Season'] = df['Apply_Month'].apply(get_season)

# B. Behavioral Features (Lags & Latency)
# "Days to Apply": How long does a user wait after signup to apply?
df['Days_to_Apply'] = (df['Apply Date'] - df['Learner SignUp DateTime']).dt.total_seconds() / (60*60*24)
df['Days_to_Apply'] = df['Days_to_Apply'].apply(lambda x: max(0, x)) # Fix negatives

# "Time of Day": Behavioral binning
def get_time_of_day(hour):
    if 5 <= hour < 12: return 'Morning'
    elif 12 <= hour < 17: return 'Afternoon'
    elif 17 <= hour < 21: return 'Evening'
    else: return 'Night'
df['Time_of_Day'] = df['Apply_Hour'].apply(get_time_of_day)

# C. Application Frequency Indicators
# "User Momentum": Count of previous applications by this specific user
# (Using Name + DOB as a proxy for unique User ID)
df['User_ID_Proxy'] = df['First Name'].astype(str) + "_" + df['Date of Birth'].astype(str)
df['User_Prev_App_Count'] = df.groupby('User_ID_Proxy').cumcount()

# "Previous Day Volume": How busy was the platform yesterday? (Lagged variable for forecasting)
# We aggregate by day, shift by 1, and merge back.
daily_counts = df.groupby(df['Apply Date'].dt.date).size().shift(1).rename('Prev_Day_App_Volume')
df['Apply_Date_Only'] = df['Apply Date'].dt.date
df = df.merge(daily_counts, left_on='Apply_Date_Only', right_index=True, how='left')
df['Prev_Day_App_Volume'] = df['Prev_Day_App_Volume'].fillna(0) # Handle first day

# D. Grouped Categories (Reducing Cardinality)
# Group Majors into Top 5 + Other
top_majors = df['Current/Intended Major'].value_counts().nlargest(5).index
df['Major_Category'] = df['Current/Intended Major'].apply(lambda x: x if x in top_majors else 'Other')

# Group Countries into Top 5 + Other
top_countries = df['Country'].value_counts().nlargest(5).index
df['Country_Group'] = df['Country'].apply(lambda x: x if x in top_countries else 'Other')

# Save the engineered dataset
df.to_csv('engineered_slu_data.csv', index=False)
print("Feature Engineering Complete. File saved as 'engineered_slu_data.csv'")
print(df[['Apply_Week', 'Season', 'Days_to_Apply', 'User_Prev_App_Count', 'Prev_Day_App_Volume']].head())

Feature Engineering Complete. File saved as 'engineered_slu_data.csv'
     Apply_Week Season  Days_to_Apply  User_Prev_App_Count  \
940          40   Fall            0.0                    0   
302          40   Fall            0.0                    0   
412          41   Fall            0.0                    0   
46           41   Fall            0.0                    0   
957          41   Fall            0.0                    0   

     Prev_Day_App_Volume  
940                  0.0  
302                  0.0  
412                  2.0  
46                   2.0  
957                  2.0  
